In [1714]:
import sys
!{sys.executable} -m pip install scipy

  Using cached scipy-1.5.2-cp37-cp37m-manylinux1_x86_64.whl (25.9 MB)


In [1717]:
from ipywidgets import HBox, VBox, Layout, Button, HTML, Label
from IPython.display import Image
from rgrok import rgrok
from numpy.random import choice
import numpy as np
import itertools
import json
from scipy.special import softmax

In [89]:
all_urls = []

In [512]:
target = ""

In [513]:
after = ""

In [505]:
response = rgrok(target, after)
after = response["last"]
urls = response["urls"]
# all_urls.extend(urls)
# urls[:2]

In [509]:
box_layout = Layout(flex_flow="row wrap")
HBox([HTML(f'<img src="{url}" style="line-height: 0; width: 100px;"/>') for url in urls], layout=box_layout)

HBox(layout=Layout(flex_flow='row wrap'))

In [970]:
queue_urls = all_urls

In [971]:
keep_urls = []

In [999]:
def update_random_url():
    global random_url
    random_url = choice(all_urls)
update_random_url()
last_urls = []

def progress_text():
    return f"{len(keep_urls)}/{len(queue_urls)}"

delete_button = Button(
    description='Delete',
    disabled=False,
    button_style='',
    icon='times')
# delete_button.on_click(lambda: all_urls.remove(random_url))
next_button = Button(
    description="Keep",
    disabled=False,
    icon="check"
)
back_button = Button(
    description="Back",
    disabled=False,
    icon="arrow-left"
)
shuffle_button = Button(
    description="Shuffle",
    disabled=False,
    icon="random"
)

url_label = Label(random_url)
progress_label = Label(progress_text())

def img_html(url):
    return f'<img src="{url}" style="line-height: 0; height: 150px;"/>'

img_1 = HTML(img_html(random_url))

def set_url(url):
    img_1.value = img_html(url)
    url_label.value = url

def shuffle():
    update_random_url()
    progress_label.value = progress_text()
    set_url(random_url)
    
def next_url():
    global queue_urls
    queue_urls.remove(random_url)
    last_urls.append(random_url)
    shuffle()

def back():
    if len(last_urls) == 0:
        return
    
    back_url = last_urls.pop()
    set_url(back_url)
    keep_urls.remove(back_url)
    queue_urls.append(back_url)
    progress_label.value = progress_text()
    

def keep_and_next():
    global keep_urls
    keep_urls.append(random_url)
    next_url()
    
next_button.on_click(lambda _: keep_and_next())

delete_button.on_click(lambda _: next_url())
back_button.on_click(lambda _: back())
shuffle_button.on_click(lambda _: shuffle())

# display(VBox([HBox([next_button, delete_button]), HBox([back_button, shuffle_button]), url_label, progress_label, img_1]))

#with next_button.hold_trait_notifications():
    #img_1.value = img_html(random_url)
    #print(random_url)

ValueError: 'a' cannot be empty unless no samples are taken

Dump into a file

In [994]:
with open("queue_urls.json", "w") as queue_file:
    json.dump(queue_urls, queue_file, indent=4)

In [995]:
with open("filtered_urls.json", "w") as unordered_file:
    json.dump(keep_urls, unordered_file, indent=4)

In [947]:
# with open("all_urls.json") as unordered_file:
#     all_urls = json.load(unordered_file)

In [998]:
# [print(url) for url in keep_urls];

In [1242]:
from multiprocessing import Process, Queue
import time

In [1329]:
sort_queue = Queue()

In [1330]:
sort_choice_queue = Queue()

In [1331]:
sort_response_queue = Queue()

In [1699]:
class CategorizedUrl:
    def __init__(self, url):
        self.url = url
        self.score = 0
    def __gt__(self, b):
        sort_queue.put(self)
        sort_queue.put(b)
        # print("waiting for sort")
        while sort_choice_queue.empty():
            time.sleep(0.05)
        return sort_choice_queue.get()
    def __lt__(self, b):
        sort_queue.put(b)
        sort_queue.put(self)
        # print("waiting for sort")
        while sort_choice_queue.empty():
            time.sleep(0.05)
        return sort_choice_queue.get()
    def __eq__(self, b):
        return False

In [1700]:
wrapped_urls = [CategorizedUrl(u) for u in keep_urls]

In [1507]:
wrapped_urls_sample = choice(wrapped_urls, size=50)

In [1508]:
def f(urls):
    sort_response_queue.put(sorted(urls))
    # [print(url) for url in urls]

In [1509]:
p = Process(target=f, args=(wrapped_urls_sample,))
p.start()

In [1510]:
# p.kill()

In [1511]:
p.is_alive()

True

In [1512]:
sorted_list = sort_queue.get()

In [1711]:
def sigmoid(offset, height, width_factor):
    return lambda x: height / (1 + np.e ** (offset + width_factor * x))

In [1811]:
right_vote_button = Button(
    description="Right",
    disabled=False,
    icon="arrow-right"
)
left_vote_button = Button(
    description="Left",
    disabled=False,
    icon="arrow-left"
)

url_view_probability_sigmoid = sigmoid(3, 6, -0.5)

def img_html(url, height = 150):
    return f'<img src="{url}" style="line-height: 0; height: {height}px;"/>'

def top_n_images():
    return [VBox([HTML(img_html(u.url, 80)), Label(str(u.score))]) for u in sorted(wrapped_urls, key=lambda w: w.score, reverse=True)[:10]]

img_vote_1 = HTML()
img_vote_2 = HTML()
max_prob_label = Label()
min_prob_label = Label()

top_n_images_box = HBox(top_n_images())

def set_vote_urls(url_1, url_2):
    img_vote_1.value = img_html(url_1)
    img_vote_2.value = img_html(url_2)

def update_vote_images():
    global vote_url_1, vote_url_2
    #try:
    avg_score = sum([u.score for u in wrapped_urls]) / len(wrapped_urls)
    probabilities = [url_view_probability_sigmoid(u.score - avg_score) + 0.1 for u in wrapped_urls]
    probabilities = softmax(probabilities)
    max_prob_label.value = str(round(max(probabilities), 10) * 100) + "%"
    min_prob_label.value = str(round(min(probabilities), 10) * 100) + "%"
    vote_url_1, vote_url_2 = choice(wrapped_urls, size=2, replace=False, p=probabilities)
    set_vote_urls(vote_url_1.url, vote_url_2.url)
    #except:
    top_n_images_box.children = top_n_images()
    #    print("none found")


def left_vote():
    vote_url_1.score += 1
    vote_url_2.score = max(vote_url_2.score - 1, 0)
    update_vote_images()

def right_vote():
    vote_url_1.score = max(vote_url_1.score - 1, 0)
    vote_url_2.score += 1
    update_vote_images()
    

left_vote_button.on_click(lambda _: left_vote())
right_vote_button.on_click(lambda _: right_vote())

update_vote_images()

display(top_n_images_box)
display(max_prob_label, min_prob_label, VBox([HBox([left_vote_button, right_vote_button]), HBox([img_vote_1, img_vote_2])]))

Label(value='5.690284999999999%')

Label(value='0.01690253%')

In [1775]:
top_urls = sorted(wrapped_urls, key=lambda w: w.score, reverse=True)

In [1795]:
top_urls[0].score

38

In [1818]:
HBox([VBox([HTML(img_html(u.url, 120)), Label(str(u.score))]) for u in top_urls[4:10]])

In [1482]:
keep_urls.index(img_1.value.split('"')[1])

2301

In [1501]:
keep_urls.index(img_2.value.split('"')[1])

2455

In [1381]:
len(keep_urls)

2456

In [1825]:
export_game = [{"score": u.score, "url": u.url} for u in sorted(wrapped_urls, key=lambda w: w.score, reverse=True)]

In [1830]:
with open('exported-game.json', 'w') as export_file:
    json.dump(export_game, export_file, indent=4)